### 11.1 フレームワークの利用

### ライブラリ・環境設定

#### ライブラリ導入

In [ ]:
# 必要ライブラリ追加導入
!pip install japanize-matplotlib -qq
!pip install torchviz -qq
!pip install torchinfo -qq

#### ライブラリインポート

In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchinfo import summary
from tqdm.notebook import tqdm

#### 環境設定

In [ ]:
np.set_printoptions(formatter={'float': '{:0.3f}'.format})
pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings('ignore')

#### GPU存在チェック

In [ ]:
# GPU存在チェック

# GPUが利用可能かどうかのチェック
device = torch.device("cuda:0" \
if torch.cuda.is_available() else "cpu")

# 利用可能な場合は"cuda:0"が出力される
print(device)

### データ準備

#### データローダー構築

In [ ]:
# データローダー構築
def get_data_loaders(batch_size=100, data_dir="./data"):
    """MNISTの訓練・テストデータをDataLoaderで返す"""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_set = datasets.MNIST(root=data_dir, train=True, download=True, transform=transform)
    test_set  = datasets.MNIST(root=data_dir, train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False, pin_memory=True)

    return train_loader, test_loader

batch_size = 100
train_loader, test_loader = get_data_loaders(batch_size)

### モデル構築

#### モデル定義

In [ ]:
# モデル定義

class Net(nn.Module):
    """全結合1層のシンプルなNN（MNIST想定: 28*28 -> 100 -> 10）"""
    def __init__(self, n_input=28*28, n_hidden=100, n_output=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_input, n_hidden),
            nn.ReLU(inplace=True),
            nn.Linear(n_hidden, n_output)
        )

    def forward(self, x):
        # (B, 1, 28, 28) -> (B, 784)
        x = torch.flatten(x, 1)
        return self.net(x)

#### 訓練用関数

In [ ]:
# 学習用関数

def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            preds = outputs.argmax(1)
            running_loss += loss.item() * labels.size(0)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

#### 検証用関数

In [ ]:
# 検証用関数

@torch.no_grad()
def validate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        preds = outputs.argmax(1)

        running_loss += loss.item() * labels.size(0)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

#### 学習関数

In [ ]:
def fit(*, n_hidden=100, num_epochs=20, lr=0.01, optimizer_class=optim.SGD, seed=42):
    """
    引数（すべてキーワード専用）:
        n_hidden        : 中間層ユニット数
        num_epochs      : 学習エポック数
        lr              : 学習率
        optimizer_class : 例) optim.SGD / optim.Adam
        seed            : 乱数シード

    返り値:
        model, history(np.ndarray: [epoch, train_loss, train_acc, val_loss, val_acc])

    依存（グローバル）:
        train_loader, test_loader, device
    """
    torch.manual_seed(seed)
    np.random.seed(seed)

    model = Net(n_hidden=n_hidden).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer_class(model.parameters(), lr=lr)

    history = []
    for epoch in tqdm(range(1, num_epochs + 1), desc="Training"):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, criterion, optimizer)
        va_loss, va_acc = validate(model, test_loader, criterion)

        print(
            f"Epoch [{epoch}/{num_epochs}] "
            f"train_loss: {tr_loss:.5f}, train_acc: {tr_acc:.5f}, "
            f"val_loss: {va_loss:.5f}, val_acc: {va_acc:.5f}"
        )
        history.append([epoch, tr_loss, tr_acc, va_loss, va_acc])

    return model, np.array(history, dtype=float)

#### 学習

In [ ]:
model1, history1 = fit(num_epochs=20, lr=0.01, optimizer_class=optim.SGD)

### 結果確認

#### 学習曲線の描画

In [ ]:
def plot_learning_curves(history, title_suffix=""):
    """損失と精度の学習曲線を並べて描画"""
    epochs = history[:, 0]
    train_loss = history[:, 1]
    train_acc  = history[:, 2]
    val_loss   = history[:, 3]
    val_acc    = history[:, 4]

    plt.figure(figsize=(8, 4), tight_layout=True)

    # --- 損失曲線 ---
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, label='訓練', color='b')
    plt.plot(epochs, val_loss, label='テスト', color='k', linestyle='--')
    plt.xlabel('繰り返し回数')
    plt.ylabel('損失')
    plt.title(f'学習曲線（損失）{title_suffix}')
    plt.legend()
    plt.grid(True)

    # --- 精度曲線 ---
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, label='訓練', color='b')
    plt.plot(epochs, val_acc, label='テスト', color='k', linestyle='--')
    plt.xlabel('繰り返し回数')
    plt.ylabel('精度')
    plt.title(f'学習曲線（精度）{title_suffix}')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_learning_curves(history1)

In [ ]:
model2, history2 = fit(num_epochs=50, lr=0.01, optimizer_class=optim.SGD)
plot_learning_curves(history2)

### バージョン確認

In [ ]:
!pip install watermark -qq
%load_ext watermark
%watermark --iversions